In [ ]:
##################################################################################################
# Author：Yun Zheng
# Email：18848234471@163.com
# If there are any shortcomings, please forgive me,because this is my first time sharing.
##################################################################################################
# Data Augmentation - Tablet
# package
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Reshape, Flatten, Input
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random
from keras import regularizers
from skimage.metrics import structural_similarity as ssim
import os
from keras.layers import BatchNormalization
##################################################################################################
# Random seed
np.random.seed(30)
random.seed(30)
tf.random.set_seed(30)
# Define directories for saving data and images
save_dir = '30_1000_0.006'
os.makedirs(save_dir, exist_ok=True)
# load your dataset
df = pd.read_csv('tablet-3-4.csv', header=None)
X1 = df.iloc[1:81, 1:].to_numpy()
# normalize
scaler = MinMaxScaler()
scaled_X1 = scaler.fit_transform(X1)
# build the generator model
generator = Sequential([
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.006)),
    BatchNormalization(),  
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.006)),
    BatchNormalization(),  
    Dense(404, activation='sigmoid', kernel_regularizer=regularizers.l2(0.006)),  
    Reshape((404, ))  
])
# compile the discriminator model
generator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0005, beta_1=0.5), metrics=['accuracy'])
# build the discriminator model
discriminator = Sequential([
    Flatten(input_shape=(404, )),
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.006)),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.006)),
    Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.006))
])
# compile the discriminator model
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0005, beta_1=0.5), metrics=['accuracy'])
# combine discriminator and generator
discriminator.trainable = False  
gan_input = Input(shape=(404,))
generated_sample = generator(gan_input)
gan_output = discriminator(generated_sample)
gan = Model(gan_input, gan_output)
# compile the Generative Adversarial Network
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0005, beta_1=0.5))
# train the GAN model
batch_size = 80
epochs = 5001
for epoch in range(epochs):
    # generate fake samples
    noise = np.random.rand(batch_size, 404)
    generated_samples = generator.predict(noise)
    # randomly select real samples
    idx = np.random.randint(0, scaled_X1.shape[0], batch_size)
    real_samples = scaled_X1[idx]
    if len(real_samples) < batch_size:
        repeats = int(np.ceil(batch_size / len(real_samples)))
        real_samples = np.repeat(real_samples, repeats, axis=0)[:batch_size]
    # train the discriminator
    d_loss_real = discriminator.train_on_batch(real_samples, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(generated_samples, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    # train the generator
    noise = np.random.rand(batch_size, 404)
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
    # output results after each iteration
    print(f"Epoch: {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")

    # save generated data and images every 100 iterations
    if epoch % 100 == 0:
        generated_samples1 = generator.predict(np.random.rand(1600, 404))
        generated_samples2 = scaler.inverse_transform(generated_samples1.reshape(1600, 404))
        S1 = generated_samples2
        # save generated data
        np.save(os.path.join(save_dir, f'generated_samples_epoch_{epoch}.npy'), S1)
        # visualize generated sample data
        P_subset = X1[:, :2]
        Q_subset = S1[:, :2]
        # visualize generated data
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.scatter(P_subset[:, 0], P_subset[:, 1], c='b', label='Original Data (P)')
        plt.title('Original Data (P)')
        plt.xlabel('Feature 1')
        plt.ylabel('Feature 2')
        plt.legend()
        plt.subplot(1, 2, 2)
        plt.scatter(Q_subset[:, 0], Q_subset[:, 1], c='r', label='Generated Data (Q)')
        plt.title('Generated Data (Q)')
        plt.xlabel('Feature 1')
        plt.ylabel('Feature 2')
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, f'generated_samples_plot_epoch_{epoch}.png'))
        plt.close()
        # visualize data
        plt.figure(dpi=100)
        mean_1 = np.mean(X1, axis=0)
        mean_2 = np.mean(S1, axis=0)
        plt.plot(mean_1, 'r')
        plt.plot(mean_2, 'b')
        plt.savefig(os.path.join(save_dir, f'mean_comparison_epoch_{epoch}.png'))
        plt.close()
# The way to get s2 just the same as s1
###################################################################################################
###################################################################################################
###################################################################################################